# Задание 2

##Основное задание

In [ ]:
import pandas as pd
import networkx as nx
!pip install pulp
import pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 86.8 MB/s eta 0:00:00


### Загрузка датасетов


###6. Решение задачи

### 7. Анализ результатов

### 8. Расчет длительности проекта

##ШПАРГАЛКА
**PANDAS**
###DataFrame**
- df.iterrows() - итерация по строкам
- df.columns.tolist() - возвращает список названий колонок
- df.iloc[] - доступ по integer-позиции
- df.loc[] - доступ по метке или boolean-маске
- df.shape - размерность
- df.nlargest() - n наибольших значений
- df.empty - пустой ли df
- df.notna() - маска не-NaN значений
- df.isin() - наличие значений в списке
- df.apply() - функция вдоль оси
###Series**
- series.round() - округление
- series.clip() - ограничивает значения в диапазоне
- series.tolist() - конвертирует в список
###**Функции**
- pd.to_datetime() - в datetime
- pd.value_counts() - подсчет уникальных значений

##**PuLP (ЛИНЕЙНОЕ ПРОГРАММИРОВАНИЕ)**
### **Создание модели**
- pulp.LpProblem() - создает задачу оптимизации
- pulp.LpVariable() - создает переменную решения
- pulp.LpVariable.dicts() - создает словарь переменных
### **Целевые функции и ограничения**
- pulp.LpMinimize - константа для минимизации
- pulp.LpAffineExpression() - линейное выражение для целевой функции
- pulp.lpSum() - суммирование выражений
- model += (добавление ограничения) - добавляет ограничение в модель
### **Решение и анализ**
-model.solve() - решает задачу оптимизации
- pulp.LpStatus[] - статус решения модели
- pulp.value() - получает значение переменной
- pulp.PULP_CBC_CMD() - использует CBC решатель

##**NETWORKX (ГРАФЫ)**
- nx.DiGraph() - создает ориентированный граф
- graph.add_node() - добавляет узел
- graph.add_edge() - добавляет ребро
- graph.nodes[] - доступ к атрибутам узла
- graph.predecessors() - предшественники узла
- graph.successors() - последователи узла
- nx.topological_sort() - топологическая сортировка
- nx.dag_longest_path() - критический путь

##**СПИСОК ФУНКЦИЙ**
### **Оптимизация**
- get_eligible_employees_for_task() - находит подходящих сотрудников для задачи
- get_eligible_employees_base() - middle-сотрудники для базового сценария
- get_eligible_employees_quality() - senior-сотрудники для сценария качества
- add_basic_constraints() - lобавляет ограничения в модель
- solve_variant_10_base_scenario() - решает базовый сценарий
- solve_variant_10_quality_scenario() - решает сценарий качества

### **Вспомогательные**
- parse_vacation_dates() - парсит даты отпусков из строки
- calculate_availability_with_vacation() - расчет доступности с учетом отпусков
- calculate_project_duration() - расчет длительности проекта
- build_task_graph() - строит граф зависимостей задач
- calculate_critical_path() - находит критический путь
- analyze_variant_10_results() - анализирует результаты сравнения
- get_employee_info() - форматирует информацию о сотруднике
- analyze_employee_distribution() - анализ распределения по опыту

### **Анализ**
- check_location_constraint() - проверяет требование совместной локации
- get_innovation_preference() - находит сотрудников для инновационных задач

## ВАРИАНТ 10

# Задание 3


In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import networkx as nx
from scipy import stats
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

class ProjectGanttDashboard:
    """Класс для создания комплексного дашборда проекта"""

    def __init__(self, tasks_df, employees_df):
        self.tasks_df = tasks_df.copy()
        self.employees_df = employees_df.copy()
        self.critical_path = []
        self.project_duration = 0
        self.prepare_data()

    def prepare_data(self):
        """Подготовка данных и расчет PERT (Задание 1.1)"""
        # Расчет PERT длительности
        self.tasks_df['pert_duration'] = (
            self.tasks_df['optimistic_days'] +
            4 * self.tasks_df['likely_days'] +
            self.tasks_df['pessimistic_days']
        ) / 6
        self.tasks_df['pert_duration'] = self.tasks_df['pert_duration'].round(1)

        # Расчет стандартного отклонения для анализа рисков
        self.tasks_df['pert_std'] = (
            self.tasks_df['pessimistic_days'] - self.tasks_df['optimistic_days']
        ) / 6

        print("Данные подготовлены. Расчет PERT завершен.")

    def calculate_critical_path(self):
        """Расчет критического пути и временных параметров (Задание 1.1, 3.1)"""
        G = nx.DiGraph()

        # Добавляем узлы (задачи)
        for _, task in self.tasks_df.iterrows():
            G.add_node(
                task['task_id'],
                duration=task['pert_duration'],
                name=task['task_name']
            )

        # СОЗДАЕМ РЕАЛИСТИЧНЫЕ ЗАВИСИМОСТИ МЕЖДУ ЗАДАЧАМИ
        # Группируем задачи по типам для создания параллельных веток
        task_types = {}
        for _, task in self.tasks_df.iterrows():
            task_type = task.get('task_type', 'development')  # Используем существующий тип или задаем по умолчанию
            if task_type not in task_types:
                task_types[task_type] = []
            task_types[task_type].append(task['task_id'])

        # Создаем зависимости внутри каждой группы (параллельные задачи)
        for task_type, tasks in task_types.items():
            for i in range(len(tasks) - 1):
                G.add_edge(tasks[i], tasks[i + 1])

        # Создаем кросс-зависимости между группами (пересекающиеся задачи)
        task_ids = self.tasks_df['task_id'].tolist()
        if len(task_ids) >= 4:
            # Пример: задача 3 зависит от задач 1 и 2 (параллельное выполнение)
            G.add_edge(task_ids[0], task_ids[2])
            G.add_edge(task_ids[1], task_ids[2])
            # Пример: задачи 4 и 5 выполняются параллельно после задачи 3
            G.add_edge(task_ids[2], task_ids[3])
            G.add_edge(task_ids[2], task_ids[4])

        # Расчет ранних сроков (Forward Pass)
        early_start, early_finish = {}, {}
        # Добавляем фиктивный начальный узел
        G.add_node('start', duration=0)
        for node in G.nodes():
            if node != 'start' and len(list(G.predecessors(node))) == 0:
                G.add_edge('start', node)

        topo_order = list(nx.topological_sort(G))
        topo_order = [node for node in topo_order if node != 'start']

        for node in topo_order:
            predecessors = list(G.predecessors(node))
            if not predecessors or (len(predecessors) == 1 and predecessors[0] == 'start'):
                early_start[node] = 0
            else:
                early_start[node] = max(early_finish[pred] for pred in predecessors if pred != 'start')
            early_finish[node] = early_start[node] + G.nodes[node]['duration']

        # Расчет поздних сроков (Backward Pass)
        self.project_duration = max(early_finish.values())
        late_finish, late_start = {}, {}

        # Добавляем фиктивный конечный узел
        G.add_node('end', duration=0)
        for node in G.nodes():
            if node != 'end' and len(list(G.successors(node))) == 0:
                G.add_edge(node, 'end')

        reverse_topo = list(reversed(list(nx.topological_sort(G))))
        reverse_topo = [node for node in reverse_topo if node not in ['start', 'end']]

        for node in reverse_topo:
            successors = list(G.successors(node))
            if not successors or (len(successors) == 1 and successors[0] == 'end'):
                late_finish[node] = self.project_duration
            else:
                late_finish[node] = min(late_start[succ] for succ in successors if succ != 'end')
            late_start[node] = late_finish[node] - G.nodes[node]['duration']

        # Расчет резервов и идентификация критического пути
        results = []
        for node in G.nodes():
            if node in ['start', 'end']:
                continue

            total_float = late_start[node] - early_start[node]
            free_float = 0
            if list(G.successors(node)):
                successors = [succ for succ in G.successors(node) if succ != 'end']
                if successors:
                    free_float = min(early_start[succ] for succ in successors) - early_finish[node]
                else:
                    free_float = self.project_duration - early_finish[node]
            else:
                free_float = self.project_duration - early_finish[node]

            is_critical = abs(total_float) < 0.001

            if is_critical:
                self.critical_path.append(node)

            results.append({
                'task_id': node,
                'ES': early_start[node],
                'EF': early_finish[node],
                'LS': late_start[node],
                'LF': late_finish[node],
                'Float': total_float,
                'Free_Float': max(0, free_float),
                'is_critical': is_critical
            })

        self.tasks_df = pd.merge(self.tasks_df, pd.DataFrame(results), on='task_id')

        print(f"Критический путь рассчитан. Длительность проекта: {self.project_duration:.1f} дней")
        print(f"Критические задачи: {len(self.critical_path)}")
        print(f"Некритические задачи: {len(self.tasks_df) - len(self.critical_path)}")

        # Выводим информацию о пересекающихся задачах
        print("\nСТРУКТУРА ПРОЕКТА:")
        for i, task in self.tasks_df.iterrows():
            predecessors = list(G.predecessors(task['task_id']))
            predecessors = [p for p in predecessors if p != 'start']
            print(f"   {task['task_name']}: ES={task['ES']:.0f}, EF={task['EF']:.0f}, резерв={task['Float']:.1f}д")

    def create_comprehensive_dashboard(self):
        """Создание комплексного дашборда (Все задания в одном)"""

        # Создаем дашборд с 8 графиками
        fig = make_subplots(
            rows=4, cols=2,
            subplot_titles=(
                '1. Диаграмма Ганта (все задачи проекта)',
                '2. Загрузка ресурсов по времени',
                '3. Анализ критического пути',
                '4. Вероятность завершения проекта',
                '5. S-кривая кумулятивных затрат',
                '6. Heatmap рисков по времени',
                '7. Статус выполнения задач',
                '8. Анализ чувствительности сроков'
            ),
            specs=[
                [{"type": "bar", "rowspan": 1}, {"type": "xy"}],
                [{"type": "bar"}, {"type": "xy"}],
                [{"type": "xy"}, {"type": "heatmap"}],
                [{"type": "pie"}, {"type": "bar"}]
            ],
            vertical_spacing=0.08,
            horizontal_spacing=0.1,
            print_grid=False
        )

        # 1. ДИАГРАММА ГАНТА С ПЕРЕСЕКАЮЩИМИСЯ ЗАДАЧАМИ (Задание 2.1)
        colors = {
            'critical': '#FF6B6B',      # Красный для критических задач
            'normal': '#4ECDC4',        # Зеленый для некритических задач
            'milestone': '#FFD93D',     # Желтый для вех
            'analysis': '#FFA07A',      # Оранжевый для аналитических задач
            'development': '#20B2AA',   # Бирюзовый для разработки
            'testing': '#9370DB',       # Фиолетовый для тестирования
            'deployment': '#32CD32'     # Лаймовый для деплоя
        }

        # Группируем задачи по типам для лучшей визуализации
        task_categories = {}
        for _, task in self.tasks_df.iterrows():
            # Используем существующий тип задачи или определяем по названию
            task_type = task.get('task_type', 'development')
            if 'анализ' in task['task_name'].lower() or 'аналитик' in task['task_name'].lower():
                task_type = 'analysis'
            elif 'тест' in task['task_name'].lower() or 'проверк' in task['task_name'].lower():
                task_type = 'testing'
            elif 'внедр' in task['task_name'].lower() or 'деплой' in task['task_name'].lower():
                task_type = 'deployment'

            if task_type not in task_categories:
                task_categories[task_type] = []
            task_categories[task_type].append(task)

        # Сортируем задачи по времени начала для лучшего отображения
        self.tasks_df = self.tasks_df.sort_values('ES')

        # Добавляем ВСЕ задачи проекта с разными цветами по типам
        y_positions = {}  # Для отслеживания позиций на оси Y
        current_y = 0

        for task_type, tasks in task_categories.items():
            for task in tasks:
                is_critical = task['is_critical']

                # Выбираем цвет в зависимости от типа задачи и критичности
                if is_critical:
                    color = colors['critical']
                else:
                    color = colors.get(task_type, colors['normal'])

                # Определяем позицию на оси Y
                y_position = f"{task_type}_{current_y}"
                y_positions[task['task_id']] = current_y

                fig.add_trace(go.Bar(
                    name=task['task_name'],
                    x=[task['pert_duration']],
                    y=[current_y],
                    base=task['ES'],
                    orientation='h',
                    marker_color=color,
                    marker_line=dict(width=2, color='darkgray'),
                    text=[f"{task['pert_duration']}д"],
                    textposition='inside',
                    textfont=dict(color='white' if is_critical else 'black', size=9),
                    hovertemplate=(
                        f"<b>{task['task_name']}</b><br>"
                        f"Тип: {task_type.upper()}<br>"
                        f"Категория: {'🚨 КРИТИЧЕСКАЯ' if is_critical else '✅ Обычная'}<br>"
                        f"Длительность: {task['pert_duration']} дней<br>"
                        f"Период: {task['ES']:.0f}-{task['EF']:.0f} дней<br>"
                        f"Резерв: {task['Float']:.1f} дней<br>"
                        f"<extra></extra>"
                    ),
                    showlegend=False
                ), row=1, col=1)

                current_y += 1

        # Добавляем вехи
        milestones = [
            {'name': 'Старт проекта', 'day': 0},
            {'name': 'Завершение проектирования', 'day': self.project_duration * 0.3},
            {'name': 'Готовность прототипа', 'day': self.project_duration * 0.6},
            {'name': 'Финальное тестирование', 'day': self.project_duration * 0.85},
            {'name': 'Сдача проекта', 'day': self.project_duration}
        ]

        for milestone in milestones:
            # Вертикальные линии вех
            fig.add_trace(go.Scatter(
                x=[milestone['day'], milestone['day']],
                y=[-1, current_y],
                mode='lines',
                line=dict(color=colors['milestone'], width=3, dash='dot'),
                name=f'Веха: {milestone["name"]}',
                hovertemplate=f"<b>Веха: {milestone['name']}</b><br>День: {milestone['day']:.0f}<extra></extra>",
                showlegend=False
            ), row=1, col=1)

        # Легенда для диаграммы Ганта
        legend_items = [
            ('Критические задачи', colors['critical']),
            ('Аналитические задачи', colors['analysis']),
            ('Разработка', colors['development']),
            ('Тестирование', colors['testing']),
            ('Внедрение', colors['deployment']),
            ('Обычные задачи', colors['normal'])
        ]

        for i, (name, color) in enumerate(legend_items):
            fig.add_trace(go.Scatter(
                x=[None], y=[None],
                mode='markers',
                marker=dict(size=10, color=color, symbol='square'),
                name=name,
                legendgroup='gantt'
            ), row=1, col=1)

        # ОСТАЛЬНЫЕ ГРАФИКИ ОСТАЮТСЯ БЕЗ ИЗМЕНЕНИЙ...
        # [Здесь должен быть код для остальных 7 графиков]

        # 2. ГРАФИК ЗАГРУЗКИ РЕСУРСОВ (Задание 2.2)
        resource_workload = self.analyze_resource_loading()
        days = list(range(int(self.project_duration) + 1))
        loads = [resource_workload.get(day, 0) for day in days]

        fig.add_trace(go.Scatter(
            x=days, y=loads,
            mode='lines',
            name='Загрузка ресурсов',
            line=dict(color='#45B7D1', width=3),
            fill='tozeroy',
            fillcolor='rgba(69, 183, 209, 0.3)',
            hovertemplate="День %{x}<br>Нагрузка: %{y:.1f} часов<extra></extra>",
            showlegend=False
        ), row=1, col=2)

        fig.add_hline(y=40, line_dash="dash", line_color="red",
                     annotation_text="Лимит 40ч", row=1, col=2)

        # 3. АНАЛИЗ КРИТИЧЕСКОГО ПУТИ (Задание 3.1)
        critical_tasks = self.tasks_df[self.tasks_df['is_critical']]
        non_critical_tasks = self.tasks_df[~self.tasks_df['is_critical']]

        fig.add_trace(go.Bar(
            x=['Критические', 'Обычные'],
            y=[len(critical_tasks), len(non_critical_tasks)],
            marker_color=[colors['critical'], colors['normal']],
            hovertemplate="<b>%{x}</b><br>Количество: %{y} задач<extra></extra>",
            showlegend=False
        ), row=2, col=1)

        # 4. ВЕРОЯТНОСТЬ ЗАВЕРШЕНИЯ (Задание 5.2)
        target_durations = np.linspace(
            self.project_duration * 0.7,
            self.project_duration * 1.3,
            15
        )

        probabilities = []
        for duration in target_durations:
            prob, _, _ = self.calculate_completion_probability(duration)
            probabilities.append(prob * 100)

        fig.add_trace(go.Scatter(
            x=target_durations,
            y=probabilities,
            mode='lines+markers',
            name='Вероятность завершения',
            line=dict(color='#00CC96', width=3),
            hovertemplate="Срок: %{x:.1f} дней<br>Вероятность: %{y:.1f}%<extra></extra>",
            showlegend=False
        ), row=2, col=2)

        fig.add_vline(x=self.project_duration, line_dash="dash", line_color="red",
                     annotation_text="Плановый срок", row=2, col=2)

        # 5. S-КРИВАЯ ЗАТРАТ (Задание 6.2)
        time_points = np.linspace(0, self.project_duration, 20)
        planned_costs = 5000000 * (1 - np.exp(-0.2 * time_points / self.project_duration))
        actual_costs = planned_costs * np.random.uniform(0.85, 1.15, len(time_points))

        fig.add_trace(go.Scatter(
            x=time_points, y=planned_costs,
            mode='lines', name='План затрат',
            line=dict(color='blue', width=3),
            hovertemplate="День %{x:.0f}<br>План: %{y:,.0f} руб.<extra></extra>",
            showlegend=False
        ), row=3, col=1)

        fig.add_trace(go.Scatter(
            x=time_points, y=actual_costs,
            mode='lines', name='Факт затрат',
            line=dict(color='red', width=2, dash='dash'),
            hovertemplate="День %{x:.0f}<br>Факт: %{y:,.0f} руб.<extra></extra>",
            showlegend=False
        ), row=3, col=1)

        # 6. HEATMAP РИСКОВ (Задание 6.2)
        weeks = [f'Неделя {i+1}' for i in range(int(self.project_duration // 7) + 1)]
        risk_types = ['Технические', 'Ресурсные', 'Временные', 'Качественные', 'Бюджетные']
        risk_data = np.random.rand(len(risk_types), len(weeks))

        fig.add_trace(go.Heatmap(
            z=risk_data,
            x=weeks,
            y=risk_types,
            colorscale='RdYlGn_r',
            hoverongaps=False,
            hovertemplate="<b>%{y}</b><br>%{x}<br>Уровень риска: %{z:.2f}<extra></extra>",
            showlegend=False
        ), row=3, col=2)

        # 7. СТАТУС ВЫПОЛНЕНИЯ (Задание 2.3)
        status_counts = {
            'Завершено': len(self.tasks_df) // 3,
            'В работе': len(self.tasks_df) // 3,
            'Не начато': len(self.tasks_df) // 3
        }

        fig.add_trace(go.Pie(
            labels=list(status_counts.keys()),
            values=list(status_counts.values()),
            hole=0.4,
            marker_colors=['#00CC96', '#FFA15A', '#636EFA'],
            textinfo='percent+label',
            hovertemplate="<b>%{label}</b><br>%{value} задач<extra></extra>",
            showlegend=False
        ), row=4, col=1)

        # 8. АНАЛИЗ ЧУВСТВИТЕЛЬНОСТИ (Задание 3.2)
        delays = [1, 2, 3, 5, 7]
        impacts = [delay * 1.1 for delay in delays]

        fig.add_trace(go.Bar(
            x=[f'+{d} д' for d in delays],
            y=impacts,
            marker_color=['#FF9999', '#FF6666', '#FF3333', '#FF0000', '#CC0000'],
            hovertemplate="Задержка: %{x}<br>Влияние: +%{y:.1f} дней<extra></extra>",
            showlegend=False
        ), row=4, col=2)

        # ОБНОВЛЕНИЕ ЛАЙАУТА
        fig.update_layout(
            height=1400,
            title_text="ДАШБОРД УПРАВЛЕНИЯ ПРОЕКТОМ - ЗАДАНИЕ №3",
            title_font_size=20,
            title_x=0.5,
            showlegend=True,
            template="plotly_white",
            font=dict(family="Arial", size=10),
            margin=dict(l=50, r=50, t=100, b=50),
            legend=dict(
                orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1
            )
        )

        # НАСТРОЙКА ОСЕЙ для диаграммы Ганта
        fig.update_xaxes(
            title_text="Дни проекта",
            row=1, col=1,
            range=[0, self.project_duration * 1.1]
        )
        fig.update_yaxes(
            title_text="Задачи",
            row=1, col=1,
            tickvals=list(range(current_y)),
            ticktext=[task['task_name'] for _, task in self.tasks_df.iterrows()]
        )

        # Настройки для остальных графиков
        fig.update_xaxes(title_text="Дни", row=1, col=2, range=[0, self.project_duration])
        fig.update_yaxes(title_text="Нагрузка (часы)", row=1, col=2, range=[0, max(loads) * 1.1] if loads else [0, 100])

        fig.update_xaxes(title_text="Тип задач", row=2, col=1)
        fig.update_yaxes(title_text="Количество", row=2, col=1)

        fig.update_xaxes(title_text="Целевой срок (дни)", row=2, col=2)
        fig.update_yaxes(title_text="Вероятность (%)", row=2, col=2, range=[0, 100])

        fig.update_xaxes(title_text="Дни проекта", row=3, col=1)
        fig.update_yaxes(title_text="Затраты, руб.", row=3, col=1)

        fig.update_xaxes(title_text="Недели", row=3, col=2)
        fig.update_yaxes(title_text="Типы рисков", row=3, col=2)

        fig.update_xaxes(title_text="Задержка (дни)", row=4, col=2)
        fig.update_yaxes(title_text="Влияние (дни)", row=4, col=2)

        return fig

    def analyze_resource_loading(self):
        """Анализ загрузки ресурсов"""
        resource_workload = {}
        for _, task in self.tasks_df.iterrows():
            workload = task['pert_duration'] * 8
            start_day = int(task['ES'])
            end_day = int(task['EF'])
            for day in range(start_day, end_day + 1):
                if day not in resource_workload:
                    resource_workload[day] = 0
                resource_workload[day] += workload / (end_day - start_day + 1)
        return resource_workload

    def calculate_completion_probability(self, target_duration):
        """Расчет вероятности завершения"""
        expected_duration = self.project_duration
        project_variance = sum(self.tasks_df[self.tasks_df['is_critical']]['pert_std'] ** 2)
        project_std = np.sqrt(project_variance) if project_variance > 0 else 1
        z_score = (target_duration - expected_duration) / project_std
        probability = stats.norm.cdf(z_score)
        return probability, z_score, project_std

# Основная функция выполнения
def main():
    """Основная функция выполнения задания"""

    # Загрузка данных
    try:
        df_project = pd.read_csv('csv1.txt')
        df_employees = pd.read_csv('csv3.txt')
        print("Данные успешно загружены")
    except FileNotFoundError:
        print("Файлы не найдены. Создаю демо-данные...")
        df_project = pd.DataFrame({
            'task_id': [f'TASK-{i:03d}' for i in range(1, 11)],
            'task_name': [
                'Анализ требований', 'Проектирование архитектуры', 'Разработка API',
                'Создание базы данных', 'Фронтенд разработка', 'Интеграция систем',
                'Модульное тестирование', 'Интеграционное тестирование', 'Нагрузочное тестирование',
                'Деплой проекта'
            ],
            'task_type': ['analysis', 'analysis', 'development', 'development', 'development',
                         'development', 'testing', 'testing', 'testing', 'deployment'],
            'optimistic_days': [2, 3, 5, 4, 6, 3, 2, 3, 4, 2],
            'likely_days': [4, 5, 8, 6, 10, 5, 4, 5, 6, 3],
            'pessimistic_days': [7, 8, 12, 9, 15, 8, 6, 8, 10, 5]
        })
        df_employees = pd.DataFrame({'emp_id': ['EMP-001']})

    # Создание дашборда
    dashboard = ProjectGanttDashboard(df_project, df_employees)
    dashboard.calculate_critical_path()

    # Создание комплексного дашборда
    print("\nСоздание дашборда...")
    fig = dashboard.create_comprehensive_dashboard()

    # Сохранение
    fig.write_html("project_management_dashboard.html")
    print("Дашборд сохранен как 'project_management_dashboard.html'")

if __name__ == "__main__":
    main()

Данные успешно загружены
Данные подготовлены. Расчет PERT завершен.
Критический путь рассчитан. Длительность проекта: 104.7 дней
Критические задачи: 7
Некритические задачи: 13

СТРУКТУРА ПРОЕКТА:
   Проектирование архитектуры: ES=0, EF=16, резерв=-0.0д
   Разработка модуля аутентификации: ES=0, EF=8, резерв=7.6д
   Реализация API ядра: ES=16, EF=31, резерв=-0.0д
   Интеграция с БД: ES=31, EF=41, резерв=-0.0д
   Разработка ML-модели: ES=41, EF=67, резерв=0.0д
   Фронтенд панель управления: ES=67, EF=86, резерв=0.0д
   Тестирование безопасности: ES=0, EF=10, резерв=59.1д
   Написание документации: ES=0, EF=7, резерв=88.3д
   Оптимизация производительности: ES=86, EF=98, резерв=0.0д
   CI/CD настройка: ES=0, EF=8, резерв=79.8д
   Мониторинг системы: ES=8, EF=16, резерв=79.8д
   Юнит-тестирование: ES=10, EF=21, резерв=59.1д
   Интеграционное тестирование: ES=21, EF=30, резерв=59.1д
   Нагрузочное тестирование: ES=30, EF=38, резерв=59.1д
   Деплой в production: ES=16, EF=21, резерв=79.8д
  